## Response selectivity

This notebook is used to generate example plots for Figure 5.

In [ ]:
import sys
sys.path.insert(0, "../scripts")

In [ ]:
import json
import logging
from pathlib import Path
from functools import partial

import ewave
import numpy as np
from numpy.random import default_rng
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.multitest as smt

import graphics_defaults
from core import MotifBackgroundSplitter, split_trials

rng = default_rng()

In [ ]:
dataset_dir = Path("../datasets/zebf-social-acoustical-ephys")
metadata_dir = dataset_dir / "metadata/"
response_dir = dataset_dir / "responses/"
stim_dir = dataset_dir / "stimuli"

In [ ]:
# examples BS and NS neurons
example_units = ["C104_3_1_c201", "C44_3_1_c74"]

In [ ]:
# load the response and split by motif
unit_responses = []
for unit_name in example_units:
    pprox_file = (response_dir / unit_name).with_suffix(".pprox")
    unit = json.loads(pprox_file.read_text())
    splitter = MotifBackgroundSplitter()
    responses = (
        split_trials(splitter, unit, metadata_dir)
        .reset_index()
        .rename(columns=lambda s: s.replace("-", "_"))
        .query("background_dBFS == -100 | foreground == 'silence'")
        .query("foreground != 'background'")
        .drop(["background", "foreground_dBFS", "offset"], axis=1)
    )
    responses["unit"] = unit_name
    unit_responses.append(responses)    
motifs = pd.concat(unit_responses)
motifs["n_events"] = motifs.events.fillna("").apply(len)
motifs["rate"] = (motifs.n_events / motifs.interval_end)
motif_names = ["silence"] + list(set(motifs.foreground.unique()) - {"silence"})
motifs["foreground"] = pd.Categorical(motifs.foreground, categories=motif_names, ordered=True)

In [ ]:
def rate_model(df):
    lm = smf.glm("n_events ~ foreground", data=df, family=sm.families.Poisson(), offset=np.log(df["interval_end"])).fit()
    conf_int = lm.conf_int()
    coefs = (
        pd.DataFrame({"stimulus": motif_names, 
                      "coef": lm.params, 
                      "std err": lm.bse, 
                      "pvalue": smt.multipletests(lm.pvalues, method="sidak")[1],
                      "coef_lcl": conf_int[0],
                      "coef_ucl": conf_int[1]
                     })
        .reset_index(drop=True)
        .set_index("stimulus")
    )
    coefs["responsive"] = (coefs.coef > 0) & (coefs.pvalue < 0.05)
    return coefs

In [ ]:
spike_type_colors = ["#70549B", "#FF7F0E"]
rates = motifs.set_index("unit").loc[example_units].groupby("unit").apply(rate_model)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(1, 2.2), dpi=300)
for i, unit_name in enumerate(example_units):
    coefs = rates.loc[unit_name]
    spont = coefs.loc["silence"]
    evoked = coefs.iloc[1:].sort_values(by="coef", ascending=False)
    evoked["rank"] = np.arange(evoked.shape[0]) + 1
    evoked["pred"] = np.exp(evoked.coef + spont.coef)
    responsive = evoked.loc[evoked.responsive]
    axes[i].axhline(np.exp(spont['coef']), color="black", linestyle='--')
    axes[i].plot(evoked["rank"], evoked.pred, 'o-', markersize=2.5, markerfacecolor="white", markeredgewidth=0.2, color=spike_type_colors[i])
    axes[i].plot(responsive["rank"], responsive.pred, 'o', markersize=1.5, color=spike_type_colors[i])
    axes[i].set_title(unit_name, fontsize=3, pad=0, loc="right")
    axes[i].set_ylim(- evoked.iloc[0].pred * 0.05, evoked.iloc[0].pred * 1.05)
axes[1].set_ylabel("Firing rate (Hz)")
axes[1].set_xlabel("Stimulus rank")

In [ ]:
fig.savefig(f"../figures/motif_rates.pdf")

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, sharex=True, figsize=(1.0, 1.0), dpi=300)
for i, unit_name in enumerate(example_units):
    coefs = rates.loc[unit_name]
    spont = coefs.loc["silence"]
    evoked = coefs.iloc[1:].sort_values(by="coef", ascending=False)
    evoked["rank"] = np.arange(evoked.shape[0]) + 1
    evoked["pred"] = np.exp(evoked.coef + spont.coef)
    evoked["norm"] = evoked.pred / evoked.pred.max()
    responsive = evoked.loc[evoked.responsive]
    axes.axhline(np.exp(spont['coef']) / evoked.pred.max(), linestyle='--', color=spike_type_colors[i])
    axes.plot(evoked["rank"], evoked.norm, 'o-', markersize=1.5, markerfacecolor="white", markeredgewidth=0.2, color=spike_type_colors[i])
    axes.plot(responsive["rank"], responsive.norm, 'o', markersize=1.0, color=spike_type_colors[i])
axes.set_ylabel("Normalized Firing Rate")
axes.set_xlabel("Stimulus rank")